In [1]:
import tensorflow as tf
import numpy as np
from data_gen import *
import time

In [2]:
embed_size = 100
enc_units = 100
feat_unit = 15
batch_size = 128

In [3]:
class Encoder(tf.keras.Model):
    def __init__(self, embedding_dim, enc_units, feat_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc1 = tf.keras.layers.Dense(feat_units, activation="relu", name="feature_output")

    def call(self, w, f, hidden):
        output, state = self.gru(w, initial_state=hidden)
        feat = self.fc1(f)
        return output, state, feat

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units), dtype=tf.float32)

In [4]:
encoder = Encoder(embed_size, enc_units, feat_unit, batch_size)

In [5]:
# sample_x = [np.random.rand(batch_size, 15, 20).astype(np.float64), np.random.rand(batch_size, 32).astype(np.float64)]
# sample_hidden = encoder.initialize_hidden_state()
# s = tf.cast(sample_x[0], tf.float32)
# k = tf.cast(sample_x[1], tf.float32)
# sample_output, sample_hidden, sample_feat = encoder(s, k, sample_hidden)

In [6]:
# sample_output.shape

In [7]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [8]:
# attention_layer = BahdanauAttention(10)
# attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

In [9]:
# attention_result.shape

In [10]:
# attention_weights.shape

In [11]:
class Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(28)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output, feat):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        x = tf.concat([context_vector, x, feat], axis=-1)
        x = tf.expand_dims(x, 1)
        output, state = self.gru(x, initial_state=hidden)
        output = tf.reshape(output, (-1, output.shape[2]))

        x = self.fc(output)
        return x, state, attention_weights

In [12]:
decoder = Decoder(embed_size, enc_units, batch_size)

# sample_decoder_output, _, _ = decoder(tf.random.uniform((batch_size,  29)), sample_hidden, sample_output, sample_feat)

In [13]:
# sample_decoder_output.shape

In [14]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

def loss_function(real, pred):
    loss_ = loss_object(real, pred)

    return tf.reduce_mean(loss_)

In [15]:
@tf.function
def train_step(root, dec_input, feature, target, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden, feat = encoder(root, feature, enc_hidden)

        dec_hidden = enc_hidden


        for t in range(target.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input[:, t], dec_hidden, enc_output, feat)
            loss += loss_function(target[:, t], predictions)

        batch_loss = (loss / int(target.shape[1]))
        variables = encoder.trainable_variables + decoder.trainable_variables

        gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [16]:
# data generator
dg = DataGen(data="data/wol-aligned.txt")

# length of a word
n_input_length = len(char2int)
n_steps_in = dg.max_root_len
n_steps_out = dg.max_output_len

6


In [17]:
print("Total train data: ", len(dg.words) * .7)
batch_size = 128
# number of batches to train
n_batches = int(len(dg.words) * .07 / batch_size) 

# python generator to generate training data at each request
# E.x word_matrix, feature = next(gen)
gen = dg.rnn_gen_data(batch_size=batch_size, n_batches=n_batches)

Total train data:  564292.3999999999


In [18]:
EPOCHS = 4
for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for step in range(n_batches):
        [root, dec_in, feat], y = next(gen)
        root = tf.cast(root, tf.float32)
        dec_in = tf.cast(dec_in, tf.float32)
        feat = tf.cast(feat, tf.float32)
        y = tf.cast(y, tf.float32)
        batch_loss = train_step(root, dec_in, feat, y, enc_hidden)
        total_loss += batch_loss

        if step % 1000 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     step,
                                                     batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
#     if (epoch + 1) % 2 == 0:
#         checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / n_batches))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 3.4652
Epoch 1 Batch 1000 Loss 0.0826
Epoch 1 Batch 2000 Loss 0.0202
Epoch 1 Batch 3000 Loss 0.0085
Epoch 1 Batch 4000 Loss 0.0069
Epoch 1 Loss 0.1489
Time taken for 1 epoch 722.1195387840271 sec

Epoch 2 Batch 0 Loss 0.0043
Epoch 2 Batch 1000 Loss 0.0060
Epoch 2 Batch 2000 Loss 0.0024
Epoch 2 Batch 3000 Loss 0.0022
Epoch 2 Batch 4000 Loss 0.0026
Epoch 2 Loss 0.0045
Time taken for 1 epoch 660.9024429321289 sec

Epoch 3 Batch 0 Loss 0.0024
Epoch 3 Batch 1000 Loss 0.0026
Epoch 3 Batch 2000 Loss 0.0017
Epoch 3 Batch 3000 Loss 0.0014
Epoch 3 Batch 4000 Loss 0.0015
Epoch 3 Loss 0.0033
Time taken for 1 epoch 661.1037704944611 sec

Epoch 4 Batch 0 Loss 0.0015
Epoch 4 Batch 1000 Loss 0.0023
Epoch 4 Batch 2000 Loss 0.0016
Epoch 4 Batch 3000 Loss 0.0016
Epoch 4 Batch 4000 Loss 0.0010
Epoch 4 Loss 0.0020
Time taken for 1 epoch 660.9877614974976 sec



In [19]:
# test_n_batches, test_batch_size =  int(len(dg.words) * .7 / batch_size), batch_size  
test_n_batches, test_batch_size = 30, 100 

# data generator for test data
test_gen = dg.rnn_gen_data(batch_size=test_batch_size, n_batches=test_n_batches, trainset=False)

In [20]:
def predict(infenc, infdec, inputs, n_steps, cardinality):
    # encode
    root = tf.cast(inputs[0], tf.float32)
#     dec_in = tf.cast(inputs[1], tf.float32)
    
    enc_hidden = encoder.initialize_hidden_state()
    hidden = [tf.zeros((1, 100), dtype=tf.float32)]
    feat = tf.cast(inputs[1], tf.float32)
#     print(root.shape, feat.shape)
    outputs, state, feat = encoder(root, feat, hidden)
    
    # start of sequence input
    start = [0.0 for _ in range(cardinality)]
#     start[0] = 1
    target_seq = np.array(start).reshape(1, cardinality)
    # collect predictions
    output = list()
#     state = tf.expand_dims(state, 1)
    for t in range(n_steps):
        # predict next char
        
        target_seq = tf.cast(target_seq, tf.float32)
#         print(target_seq.shape, state.shape, outputs.shape, feat.shape)
        yhat, h, att_w = infdec(target_seq, state, outputs, feat)
        # store prediction
#         print(yhat.shape)
        output.append(np.array(yhat))
        # update state
        state = h
        # update target sequence
        target_seq = yhat
    return np.stack(output)

In [21]:
# shows sample examples and calculates accuracy

total, correct = 0, 0
in_word = 0
sims = []
for b in range(test_n_batches):
    # get data from test data generator
    [X1, X2, X3], y = next(test_gen)
    for j in range(test_batch_size):
        word_features = X3[j].reshape((1, X3.shape[1])) 
        root_word_matrix = X1[j].reshape((1, X1.shape[1], X1.shape[2]))
#         word_index = X4[j].reshape((1, X4.shape[1]))
        # predicts the target word given root word and features
        
        target = predict(encoder, decoder, [root_word_matrix, word_features], n_steps_out, n_input_length)
        root = ''.join(dg.one_hot_decode(X1[j]))#.replace('&', ' ')
        word = ''.join(dg.one_hot_decode(y[j]))#.replace('&', ' ')
        targetS = ''.join(dg.one_hot_decode(target))#.replace('&', ' ')
#         sims.append(dg.word_sim(word, targetS))
        
        # checks if the predicted and the real words are equal
        if dg.one_hot_decode(y[j]) == dg.one_hot_decode(target):
            correct += 1
        else:
            print(root, word.split('&')[0], '\t\t', targetS.split('&')[0])
#         if root.strip() in targetS.strip():
#             in_word += 1
#     print(b, root, word, targetS)
    total += test_batch_size
    

print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

karchch         karchchissageetu 		 keeeketefanaona
anqqar          anqqarissaanaageetoo 		 agaonacjebenereceqaqnagabeenene
koshsh          koshshissiiddi 		 kabeknacjebaenenenneneenanenaee
aakim           aakimees 		 awage
wocam           wocamissiyageetee 		 wooyacjbeceneeeeeneneeeeeeebene
tum             tumikkii 		 tezefnejnaaaanaabecaneen jejene
qaatt           qaattsseeta 		 qeonaabeepefeqxagunefejeeee    
inott           inottssaanaageetee 		 inaonacbeeefeica'nene eeaegeena
qolett          qolettiyari 		 qaonaone e'e     jeeejeennefe
yambbar         yambbarissabeekkee 		 yeoajaj wocij                  
wuxaawux        wuxaawuxibeettennee 		 wooykeeeeee     emeeneeeeeeeeee
tikk            tikkageeti 		 tetaf                          
xeeray          xeerayissoos 		 xeoakka'nacetenaneeejenene'eeen
quf             qufissibeenna 		 qeonaagege'eeeqeneeneeeenenanee
sholloorett     sholloorettidi 		 shbaata
wobb            wobbssabeukku 		 wooje       ic bicbe'acefepaa
barchchey       

gufann          gufannssenna 		 geonaonaeeobijs   bici
dabbakk         dabbakkiyageeti 		 dajegba
bunjjibunjj     bunjjibunjjibeokkonaa 		 bubegekaobacepeonacnaayabene'na
piqq            piqqssidoro 		 pagaonekeubeeeeneneeeeeeebeneje
qem''elatt      qem''elattidoree 		 qaonaoneegeneneeneekijna'neqsih
maggulss        maggulssiyogeeti 		 majtetihwa'                    
ticicatt        ticicattabeikkinaa 		 teneinegne
dancc           danccissiyogeeta 		 dawegete
tifilatt        tifilattidoree 		 tunaonaaeena
qoobbat         qoobbatanee 		 qionagjaneenaejeneneejjeeeenene
yiillot         yiillotuutee 		 yakaibajaj                     
karshsh         karshshissidosona 		 kadekafaenannanenetagxe
darccuntt       darccunttssiyogee 		 dabebeonaoneeeefeeeeeteobienena
qaar            qaarissoppo 		 qiinaabagagb     j             
qirxxat         qirxxatennee 		 qionaonenaneeekeebeepefadanaaba
corbbobay       corbbobayidara 		 cegkenaaneeneneeeneebe
macimachch      macimachchissaree 		 majabeeeega

inggaaz         inggaazissiyora 		 inaoyaob                       
shol            sholisseetii 		 shbaene
kachachch       kachachchaanaagee 		 kanaebeoneebeeeenajeeje
xonddoqay       xonddoqayeeta 		 xeoaina
amal            amalissaanaagee 		 arainaabe
xurqqay         xurqqayissiyogeetoo 		 xeoemacre
inott           inottssiyogaa 		 inaobabeee
lu'aalu'        lu'aalu'idonaa 		 lagabeeacetetazeneganeenneeeene
lanqq           lanqqite 		 leganeineeneneeeefe
gaabbul         gaabbulibeennee 		 geokaobe
mulccet         mulccetaasi 		 majageteeteenaceteeneeeeeeeenee
ooh             oohii 		 okaadaceeeeebeejefeeeeneneenere
zilaal          zilaalidora 		 zeonntaonaj                    
pellell         pellellssikkii 		 pegne
doom            doomettaas 		 dejaabebeejjj jis              
qaxan           qaxanissido 		 qeonanabenjeeneeeneneneeejejeje
habbeer         habbeerawsu 		 hukaonaga
zerkkekk        zerkkekkssogeeti 		 zebeeteobacaceeeweeweetegfe
tunttat         tunttatissite 		 tuteona'n

wolqqantt       wolqqanttssiyagaa 		 woobaokacbe
ghsh            ghshissiyari 		 gegke
harshshuush     harshshuushiyageetee 		 hutegknneonaubageteenijjicjc jj
sheney          sheneyissogeetee 		 shbeabeeeene
godat           godatay 		 geokaobaen                     
happurssaa      happurssaaogoo 		 hurana
dombbolo''      dombbolo''ssikkinaa 		 daoenaeni'ne'baobaceteenaccegen
ashot           ashotiyagee 		 akaoxed                        
hiraag          hiraagiyara 		 hureinebeejbejejj jjj jej ji'u'
pooshsh         pooshshissokko 		 pigi
massamm         massammite 		 majaraabeeeeeeepeepeeeeteenannj
baq             baqissuppu 		 bajte
ishal           ishalissiyoree 		 ikaobaceane'ne
tamaar          tamaarissaree 		 tezafbeetanacjjeneeneeeqjjejeee
waass           waassagoo 		 woobkeeneeeneeeeegegeeeeeeeeeee
kaqq            kaqqara 		 kafejbe'pemege'eeeeenenenenenen
kimbbish        kimbbishidi 		 kakeoaanabenanbanjnabaja
bulxx           bulxxabeakka 		 becageetefnaenaeeebeebem  m    
muux

shaah           shaahissabeukku 		 shbaanaga
ginar           ginarissadee 		 geoka
gooz            goozoppo 		 ge'na
paatt           paattay 		 pagaofekenaibegeneqnanejebeexec
handdaraash     handdaraashite 		 hutegeteqbi'eqjegegne
baaliget        baaligetidetii 		 bebege
minjj           minjjidogeetu 		 magabaceteen  c jee eje        
sheeshsh        sheeshshissidara 		 shbaana
wosttaq         wosttaqissibeennee 		 wookab m      ' ' 'aceceqaqacec
puut            puutissidara 		 pagaonagajeenjjjj jeceje
xaall           xaalliyogeetoo 		 xeoafbacaeepeeeeqeeeqeqaebeenee
yaag            yaagiyoree 		 yakaibegenee
xa              xaissiyoro 		 xeormaj                        
dubbatt         dubbattibeokkonaa 		 dakeoebeenenazda
geellatt        geellattssiyogeetoo 		 gawge
poddatt         poddattssibeekketa 		 pegeenaonagnnnnnananajebezeteen
gubaay          gubaayissaro 		 geoka'neqanazebeenaeweneceeeeen
gololatt        gololattageetu 		 geopa'teqejefeeje ejeqe'eemenee
hoskk           hoskk

...

qirnnaac        qirnnaacettennee 		 qiona'neqjeqeeeeeeeneeeeeeeeeee
qitatt          qitattssageetoo 		 qagte
lokkom          lokkomidi 		 labafaabacazegne
yoot            yootissite 		 yaba
ooh             oohissidogeetu 		 okaok                          
parggagg        parggaggaanaagoo 		 pagopaobeenabe'neijecejeen     
yez             yezissogeeta 		 yanaoba
yolppay         yolppayokkonii 		 yakaobiobahjjjefeeee    'icjeob
dinnaaq         dinnaaqidageetu 		 dajegnaqbaaeteone
barott          barottsseetii 		 beeeefeenaejeneneeebejejeeeeebe
gaabbul         gaabbulissoppite 		 ga'kaonaeeeeneee
yuud            yuudissibeennee 		 yabe
kiitt           kiittoosona 		 kafeobaanecejegbeecegnefenacena
koyt            koytissidageetee 		 kekanbabeejeeqecaqittta
waallatt        waallattssiyagee 		 waobacabeeeseqihiba'ebeqe'eteox
yambbar         yambbariyaree 		 yonaijaane
shereq          sherequ 		 shnaane
coppay          coppayissidogoo 		 ceteobaanaj           ickeeneq 
amman           ammana

boqoqq          boqoqqidogeetu 		 bebekafoc mage
d               diibeokko 		 daciiegeenegenebebeeaneenenneee
aakk            aakkuuteetii 		 akaonenarefeelajijezezeqeqaii
biz''           biz''ogee 		 bejte
malaat          malaatissokkona 		 majaraqicacesbee  ceceeemenjanj
ak              akssageetoo 		 akaobaaba
geedd           geeddssagaa 		 ge'kacba
deemaar         deemaarissaanaagaa 		 dewegafagnanaga
koochch         koochchissees 		 kanaoba
pooll           poollida 		 page
be              beissiyaree 		 beeeeneuwaejeeeceeeeeebeeejefee
diipp           diippiis 		 dajegge'ejeqaejecaabanabeqe
oyq             oyqees 		 okaabagne
dagay           dagayissageetoo 		 dabejeqjacteqana''egabegenada
bochch          bochchana 		 beeefeneneenenegnnnegnagnefeeee
salall          salallssideta 		 sh
baa'ul          baa'ulissukku 		 beeeneqjeqae
shonggatt       shonggattssidogoo 		 shbaanagbe
qallab          qallabissokkona 		 qaonaabeenenenefe
shir            shirabeukku 		 shbae
coppay          

mentter         mentteriyara 		 majabeenegbeeejeneeejejeeeeeeef
mirqqett        mirqqettssikke 		 majaiakaibio     egbkepezaqiqte
baazzat         baazzatikke 		 bebekabaate
xiiximm         xiiximmoro 		 xefairaceneqaana
lanqq           lanqqiyonii 		 laga
bizhggam        bizhggamiyogeeti 		 bebeokiinagneqpeteenagbe'jjenac
munaqq          munaqqibeokkonii 		 majnaobaeenabeen jenajjjebeeeen
suntt           sunttssibeokko 		 shoaj                          
gubbis          gubbisissikkinaa 		 geukeokaadaw                   
dol''           dol''ibeokko 		 da
podday          poddayogeetu 		 pigaobecumababbeeeefegeceg cege
xambbixall      xambbixallssidogeetee 		 xebabe
mitt            mittenna 		 maoteaba
dentt           denttssidageetee 		 dabegetaceteeneonagnanenenabena
hobb            hobbido 		 hukainefeeejeujeeenee    eq ce'
zuull           zuullssido 		 zeofanaja
hallacc         hallaccissadii 		 hutajnafa
shawhet         shawhetogeetu 		 shbaajeenaeneeneqeeje e        
shiiqq        

wontt           wonttay 		 wooyk                          
darcc           darccissiyageeti 		 dabeeeeeeeeeeteqjeteeb ccegenai
magaax          magaaxissidageetu 		 majaeeeexenefaaeeebeeeeeeeeeeee
hobbatt         hobbattara 		 hugagnteoxa
hit             hitoppo 		 hageona
mus             musissaanaagoo 		 maba
hanttaarett     hanttaarettidogeetee 		 hutega
erett           erettuutee 		 etafbabbenen ceecwegw p  hwicce
hanccaqarot     hanccaqarotekkee 		 hugeonenaeneew peenefawejbeetef
injjey          injjeyissennee 		 inaobaobeneennaeaejiej c  ac  '
bun             bunaanaagoo 		 beeefeeteanabenecnnneneejefenae
qaxar           qaxarissaanaageetoo 		 qeonaonainegetennenaeeeetaonace
gilil           gililidageeta 		 gaeenagneeeetenjanajebee
ep              epaabeekkee 		 etaanabbereneennna'eeejeeezeeen
luqaaluq        luqaaluqissana 		 lagxeore
wonzz           wonzzettennee 		 wooypew                        
qijjil          qijjilidogeetoo 		 qaganai'eeeneebeeen jjeqjenenee
shukk          

kolxx           kolxxoos 		 kaneoneeeeeqeeebeeneq eeceeeqiq
mass            massssibeekketa 		 maaaefaeneneemeetefneeneqefeqje
shankkala''     shankkala''idogoo 		 shnagbekianagztefnaoneeenenejen
ayy             ayyogeetu 		 akaofaog ci
gepp            geppssagoo 		 geokagne
ceegg           ceeggiyogeeta 		 cetaonagbetaeneeeeeeeneeeeeeefe
tuqq            tuqqogoo 		 tana
ashilaal        ashilaalissiyogeeta 		 akagebaejee eeeneeeeejejeeeeeee
gaannat         gaannatissibeettenna 		 geokaobacneneenafbacbaceeeeeeee
kakkaf          kakkafekketii 		 kafekte
buzott          buzottssidoro 		 babeke
shaf            shafagee 		 shbaegeeeeeneneeeeeeeneeeneeeee
mololl          molollssaanaari 		 manabefahje
mucuraak        mucuraakibeokko 		 majajykaanaaben                
be              beissabeukku 		 beeefnenaine
gepp            geppiyoro 		 geoka
lee             leeidogeetee 		 laganeenagneteeeneebag
zooq            zooqissidi 		 zeoeteofaggee     je           
suur            suurida 		 sh
m